In [1]:
!pip install -q transformers gradio

In [2]:
import re
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

PYTHON_EXAMPLES = {
    "Reverse a string": "```python\ndef reverse_string(s):\n    return s[::-1]\n```",
    "Palindrome checker": "```python\ndef is_palindrome(s):\n    return s == s[::-1]\n```",
    "Remove duplicates from a list": "```python\ndef remove_duplicates(lst):\n    return list(set(lst))\n```"
}

def extract_code_block(text):
    """Extract the first code block from text using delimiters"""
    pattern = r'```(?:[a-zA-Z]+)?\n(.*?)\n```'
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else text

def generate_code(task, temperature=0.7, top_p=0.9, max_new_tokens=512):
    if model is None:
        return "Model not loaded properly!"

    try:
        example_text = "\n".join([f"Task: {t}\nCode:\n{c}" for t, c in PYTHON_EXAMPLES.items()])
        prompt = (
            "You are a Python code generation assistant. Your response must contain ONLY CODE with "
            "NO EXPLANATIONS, NO THOUGHTS, and NO NATURAL LANGUAGE.\n\n"
            "Example output format:\n"
            f"{example_text}\n\n"
            "Now generate Python code for this task:\n"
            f"Task: {task}\n"
            "```python\n"
        )

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
        inputs = inputs.to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,
            num_return_sequences=1
        )

        full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_part = full_response[len(prompt):]
        return extract_code_block(generated_part)

    except Exception as e:
        return f"❌ Error: {str(e)}"

with gr.Blocks() as demo:
    gr.Markdown("## 💻 Python Code Generator")

    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(
                label="Your Python Coding Request",
                placeholder="e.g., 'Reverse a linked list'",
                lines=3
            )

            with gr.Accordion("Generation Parameters", open=False):
                temperature = gr.Slider(
                    minimum=0.01,
                    maximum=1.0,
                    value=0.7,
                    step=0.01,
                    label="Temperature"
                )
                top_p = gr.Slider(
                    minimum=0.1,
                    maximum=1.0,
                    value=0.9,
                    step=0.01,
                    label="Top-p"
                )
                max_new_tokens = gr.Slider(
                    minimum=64,
                    maximum=1024,
                    value=512,
                    step=32,
                    label="Max Tokens"
                )

            btn = gr.Button("Generate Code", variant="primary")

        with gr.Column():
            code_output = gr.Code(
                label="Generated Python Code",
                language="python",
                interactive=True
            )
            with gr.Column():
                gr.Markdown("### Example Python Tasks")
                gr.Examples(
                    examples=[
                        "Reverse a string",
                        "Palindrome checker",
                        "Remove duplicates from a list",
                        "Calculate factorial",
                        "Check if number is prime"
                    ],
                    inputs=[prompt],
                    label="",
                    examples_per_page=5
                )

    btn.click(
        generate_code,
        inputs=[prompt, temperature, top_p, max_new_tokens],
        outputs=code_output
    )

try:
    demo.launch(share=True)
except:
    demo.launch(server_port=8080)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://85317699e8d14c1f67.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
